**Import bibliotek:**

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout
from tensorflow import keras

import albumentations as A
import cv2

C:\projects\Wizualizacja_danych_cz1\env\lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of AlbumentationsX (2.0.10) is available! Your version is 2.0.9. Upgrade using: pip install -U albumentationsx
  check_for_updates()


**Augumentacja:**

In [2]:
albumential_train = A.Compose([
    A.PadIfNeeded(min_height=30, min_width=30, border_mode=cv2.BORDER_REFLECT_101),
    A.RandomCrop(28, 28),
    A.Rotate(limit=8, border_mode=cv2.BORDER_REFLECT_101, p=0.2),
    A.RandomBrightnessContrast(0.1, 0.1, p=0.6)
])

**Funkcje związane z przygotowaniem zestawów danych dla TF:**

In [3]:
def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    x = tf.expand_dims(x, -1)
    return x, y

def albu_apply(img):
    if img.ndim == 2:
        img = np.expand_dims(img, -1)                 
    out = albumential_train(image=img)['image']        
    return out

def albu_map(x, y):
    x = tf.numpy_function(lambda z: albu_apply(z), [x], tf.uint8)
    x.set_shape([28, 28, 1])                       
    x = tf.cast(x, tf.float32) / 255.0             
    return x, y

**Podział danych na zbiór treningowy, walidacyjny oraz testowy:**

In [4]:
(x_train_dl, y_train_dl), (x_test_dl, y_test_dl) = tf.keras.datasets.fashion_mnist.load_data()

images = np.concatenate([x_train_dl, x_test_dl], axis=0)
labels = np.concatenate([y_train_dl, y_test_dl], axis=0)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=10, stratify=labels)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=10, stratify=y_train)

**Tworzenie Datasetów TF:**

In [5]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH = 128

train_ds = (tf.data.Dataset.from_tensor_slices((X_train, y_train))
            .shuffle(len(X_train), seed=42, reshuffle_each_iteration=True)
            .map(albu_map, num_parallel_calls=AUTOTUNE)
            .batch(BATCH)
            .prefetch(AUTOTUNE))

val_ds = (tf.data.Dataset.from_tensor_slices((X_val, y_val))
            .map(preprocess, num_parallel_calls=AUTOTUNE)
            .batch(BATCH).prefetch(AUTOTUNE))

test_ds = (tf.data.Dataset.from_tensor_slices((X_test, y_test))
           .map(preprocess, num_parallel_calls=AUTOTUNE)
           .batch(BATCH).prefetch(AUTOTUNE))

**Tworzenie Sieci CNN oraz kompilacja:**

In [6]:
f_mnist_model = tf.keras.Sequential([
      
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
        
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax'), 
])

opt = tf.keras.optimizers.AdamW(learning_rate=0.002, weight_decay=1e-4)

f_mnist_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'])

C:\projects\Wizualizacja_danych_cz1\env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Deklarowanie callbacków:**

In [7]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=12,
    restore_best_weights=True,
    verbose=1)

ckpt = ModelCheckpoint(                                                                                                                                        
    'best_model.keras', 
    
    monitor='val_accuracy', 
    save_best_only=True, 
    verbose=1)

history = f_mnist_model.fit(
    train_ds,
    validation_data= val_ds,
    epochs=120,
    callbacks=[reduce_lr, early_stop, ckpt],
    verbose=1)

Epoch 1/120
443/443 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.7261 - loss: 0.7413
Epoch 1: val_accuracy improved from -inf to 0.58730, saving model to best_model.keras
443/443 ━━━━━━━━━━━━━━━━━━━━ 67s 130ms/step - accuracy: 0.7263 - loss: 0.7409 - val_accuracy: 0.5873 - val_loss: 1.0372 - learning_rate: 0.0020
Epoch 2/120
443/443 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.8633 - loss: 0.3779
Epoch 2: val_accuracy improved from 0.58730 to 0.87270, saving model to best_model.keras
443/443 ━━━━━━━━━━━━━━━━━━━━ 57s 129ms/step - accuracy: 0.8633 - loss: 0.3779 - val_accuracy: 0.8727 - val_loss: 0.3355 - learning_rate: 0.0020
Epoch 3/120
443/443 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.8831 - loss: 0.3183
Epoch 3: val_accuracy improved from 0.87270 to 0.89016, saving model to best_model.keras
443/443 ━━━━━━━━━━━━━━━━━━━━ 59s 133ms/step - accuracy: 0.8831 - loss: 0.3183 - val_accuracy: 0.8902 - val_loss: 0.2935 - learning_rate: 0.0020
Epoch 4/120
443/443 ━━━━━━━━━━━━━━━━━

443/443 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.9468 - loss: 0.1434
Epoch 22: val_accuracy improved from 0.93905 to 0.93968, saving model to best_model.keras
443/443 ━━━━━━━━━━━━━━━━━━━━ 51s 113ms/step - accuracy: 0.9468 - loss: 0.1434 - val_accuracy: 0.9397 - val_loss: 0.1604 - learning_rate: 2.5000e-04
Epoch 23/120
443/443 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.9487 - loss: 0.1388
Epoch 23: val_accuracy did not improve from 0.93968
443/443 ━━━━━━━━━━━━━━━━━━━━ 43s 97ms/step - accuracy: 0.9487 - loss: 0.1388 - val_accuracy: 0.9392 - val_loss: 0.1637 - learning_rate: 2.5000e-04
Epoch 24/120
443/443 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9463 - loss: 0.1427
Epoch 24: val_accuracy improved from 0.93968 to 0.94206, saving model to best_model.keras
443/443 ━━━━━━━━━━━━━━━━━━━━ 59s 131ms/step - accuracy: 0.9463 - loss: 0.1427 - val_accuracy: 0.9421 - val_loss: 0.1563 - learning_rate: 2.5000e-04
Epoch 25/120
443/443 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0

**Metryki dla danych testowych:**

In [8]:
test_loss, test_acc = f_mnist_model.evaluate(test_ds, verbose=1)
print(f"Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9490 - loss: 0.1411
Test loss: 0.1424 | Test acc: 0.9506
